In [61]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
kc1 = pd.read_csv(r'/content/kc1_csv.csv')
print('kc1 shape: ', kc1.shape)
kc1.head()

kc1 shape:  (2109, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,...,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,...,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True


In [63]:
kc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                2109 non-null   float64
 1   v(g)               2109 non-null   float64
 2   ev(g)              2109 non-null   float64
 3   iv(g)              2109 non-null   float64
 4   n                  2109 non-null   float64
 5   v                  2109 non-null   float64
 6   l                  2109 non-null   float64
 7   d                  2109 non-null   float64
 8   i                  2109 non-null   float64
 9   e                  2109 non-null   float64
 10  b                  2109 non-null   float64
 11  t                  2109 non-null   float64
 12  lOCode             2109 non-null   int64  
 13  lOComment          2109 non-null   int64  
 14  lOBlank            2109 non-null   int64  
 15  locCodeAndComment  2109 non-null   int64  
 16  uniq_Op            2109 

In [64]:
target = 'defects'
y = kc1[target]
x=kc1.drop(['defects'], axis=1)
x.shape

(2109, 21)

after running " *from mlxtend.feature_selection import SequentialFeatureSelector as SFS* " this command you might get error this is because mlxtend is trying to import joblib from sklearn but it no longer exist there so fix this issue go to the file location (there will a file location link in the error message like this " /usr/local/lib/python3.7/dist-packages/mlxtend/feature_selection/sequential_feature_selector.py <module> " click on this location link)
then it will show you code written in that file then you just have to look for this line " from sklearn.externals.joblib import Parallel, delayed " and edit it to " from joblib import Parallel, delayed " and then save it by pressing ctrl+s. then run the code again. Again it will give error but this time the location link will be different and you have to again edit the file in same way as done priviouslly and save it and run the code again and this will solve the error.

In [65]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.svm import SVC
svc = SVC()


In [66]:
efs = EFS(svc, # can use any any learning method 
          min_features=1, # can set any limit from 1 to less then or equal to max_feature value
          max_features=4, # can set any value less then toatal number of features but greater then or equal to min_features
                          # higer the difference between the min and max value higher the time taken to select features
          scoring='accuracy',
          print_progress=True,
          cv=2
          ).fit(np.array(x[x.columns[0:5]].fillna(0)), y)
                                    # set value of columns like [0:x] where x is equal to value of max_features

Features: 30/30

In [67]:
efs.best_idx_

(0, 4)

In [68]:
x_reduced = efs.transform(x)
x_reduced.shape

(2109, 2)

In [69]:
y.shape

(2109,)

In [70]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [71]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2109, 21)
(2109,)
-------------------
(1476, 2)
(1476,)
(633, 2)
(633,)


In [72]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8451816745655608
Th confusion matrix of SVM:
  [[523   8]
 [ 90  12]]


In [73]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[523   8]
 [ 90  12]]
              precision    recall  f1-score   support

       False       0.85      0.98      0.91       531
        True       0.60      0.12      0.20       102

    accuracy                           0.85       633
   macro avg       0.73      0.55      0.56       633
weighted avg       0.81      0.85      0.80       633



In [74]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8593996840442338
The confusion matrix of KNN:
  [[522   9]
 [ 80  22]]
[[522   9]
 [ 80  22]]
              precision    recall  f1-score   support

       False       0.87      0.98      0.92       531
        True       0.71      0.22      0.33       102

    accuracy                           0.86       633
   macro avg       0.79      0.60      0.63       633
weighted avg       0.84      0.86      0.83       633



In [75]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.8325434439178515
[[485  46]
 [ 60  42]]
              precision    recall  f1-score   support

       False       0.89      0.91      0.90       531
        True       0.48      0.41      0.44       102

    accuracy                           0.83       633
   macro avg       0.68      0.66      0.67       633
weighted avg       0.82      0.83      0.83       633



In [76]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8434959349593496

In [77]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.87      0.95      0.91       531
        True       0.53      0.26      0.35       102

    accuracy                           0.84       633
   macro avg       0.70      0.61      0.63       633
weighted avg       0.82      0.84      0.82       633

[[507  24]
 [ 75  27]]
